In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# test Robust

In [69]:
df_1 = pd.DataFrame(np.random.normal(size=(500_000, 3)),
                   columns=["metric_val1", "metric_val2", "metric_val3"])
df_1['label'] = 'A'
df_2 = pd.DataFrame(np.random.normal(size=(500_000, 3)),
                   columns=["metric_val1", "metric_val2", "metric_val3"])
df_2['label'] = 'B'
df_3 = pd.DataFrame(np.random.normal(size=(500_000, 3)),
                   columns=["metric_val1", "metric_val2", "metric_val3"])
df_3['label'] = 'C'

In [70]:
res = pd.concat([df_1, df_2, df_3])

In [96]:
def check_target_type(
    dataframe,
    column,
):
    """
    Check type of target: binary / non-binary.
    """
    unique_vals_count = dataframe[column].nunique()
    if unique_vals_count > 2:
        return "non-binary"
    unique_vals = set(dataframe[column].unique())
    if unique_vals == {0, 1} or unique_vals in {0, 1}:
        return "binary"
    return "non-binary"

In [97]:
def check_target_type_new(
    dataframe,
    column,
):
    """
    Check type of target: binary / non-binary.
    """
    unique_vals = set(dataframe[column].values)
    if unique_vals == {0, 1}:
        return "binary"
    else:
        return "non-binary"

In [105]:
%%time
check_target_type(res, 'metric_val4')

CPU times: user 17.7 ms, sys: 3.25 ms, total: 21 ms
Wall time: 19.4 ms


'binary'

In [39]:
from ambrosia.tester import Tester

In [40]:
tester = Tester()

In [57]:
0.249216 * 3

0.747648

In [56]:
tester.run(dataframe=res, metrics=['metric_val1'], column_groups='label')

,first_type_error,pvalue,effect,confidence_interval,metric name,group A label,group B label
0,0.016667,0.747649,0.023052,"(-0.0324, 0.0785)",metric_val1,A,B
1,0.016667,2.837958,0.001352,"(-0.054, 0.0567)",metric_val1,A,C
2,0.016667,0.826893,-0.021700,"(-0.0769, 0.0335)",metric_val1,B,C


In [59]:
tester.run(dataframe=res[(res['label'] =='A') | (res['label'] =='B')], metrics=['metric_val1'], column_groups='label')

,first_type_error,pvalue,effect,confidence_interval,metric name,group A label,group B label
0,0.005556,0.249216,0.023052,"(-0.0324, 0.0785)",metric_val1,A,B


In [58]:
tester.run(dataframe=res[(res['label'] =='A') | (res['label'] =='B')], metrics=['metric_val1', 'metric_val2'], column_groups='label')

,first_type_error,pvalue,effect,confidence_interval,metric name,group A label,group B label
0,0.005556,0.249216,0.023052,"(-0.0324, 0.0785)",metric_val1,A,B
1,0.005556,0.142035,-0.029260,"(-0.0845, 0.026)",metric_val2,A,B


In [22]:
import joblib

In [24]:
model = joblib.load('tests/configs/mlvar_reducer_model.pkl')

In [28]:
trans = model.predict(table)

In [29]:
np.mean(trans)

1683.6826818075872

In [30]:
np.var(trans)

86920.62715058296

In [33]:
table.target.mean()

1684.050907878038

In [34]:
table.target.var()

955119.6580605546

In [16]:
import numpy as np
import pandas as pd

In [2]:
from ambrosia.preprocessing import AggregatePreprocessor
from ambrosia.preprocessing import Preprocessor

In [3]:
kek = AggregatePreprocessor()

In [4]:
inspect.getargspec(f)[0]

NameError: name 'inspect' is not defined

In [4]:
inspect.signature(kek.fit)

NameError: name 'inspect' is not defined

In [6]:
getattr(sys.modules[__name__], 'Preprocessor')

NameError: name 'sys' is not defined

In [ ]:
eval()

In [25]:
eval('Preprocessor')

ambrosia.preprocessing.preprocessor.Preprocessor

In [21]:
.__class__.__name__

SyntaxError: invalid syntax (2760301575.py, line 1)

In [5]:
AggregatePreprocessor()

In [26]:
table: pd.DataFrame = pd.read_csv("tests/test_data/nonlin_var_table.csv")

In [21]:
table[['feature_1', 'feature_2']].cov()

,feature_1,feature_2
feature_1,4.332891,0.637960
feature_2,0.637960,99.311681


In [7]:
transformer = Preprocessor(table, verbose=False)
transformed = (
    transformer.robust("feature_1", alpha=0.01, tail="right")
    .iqr(["feature_2", "feature_3"])
    .iqr(["feature_1"])
    .log("feature_1")
    .boxcox(["feature_2", "feature_3"])
    .cuped("target", "feature_3", name="target_cuped")
    .data()
)

      feature_1  feature_2  feature_3       target
1      1.807905   1.775187   6.774050  2300.801046
2      1.830794   3.363904   4.872551   869.573677
3      1.160506   3.447586   2.887106   157.878257
4      1.575655   3.110634   5.163880  1020.129064
5      1.104943   2.044608   6.420628  1942.369042
...         ...        ...        ...          ...
2995   1.363962   2.932011   7.366038  2903.761007
2996   0.431863   1.704545   5.436949  1168.389537
2997   1.309711   2.341441   8.172211  3883.220463
2998   0.886774   1.263778   5.950709  1543.097011
2999   0.944045   2.174664   4.803001   796.584377

[2774 rows x 4 columns]
['feature_3']


In [8]:
transformer.store_transformations("tests/configs/preprocessor_config.json")

In [9]:
transformer = Preprocessor(table, verbose=False)

In [10]:
import json

In [17]:
load_path = "tests/configs/preprocessor_config.json"

In [12]:
with open(load_path, "r+") as file:
    params = json.load(file)

In [13]:
params

{'RobustPreprocessor1': {'tail': 'right',
  'column_names': ['feature_1'],
  'alpha': [0.01],
  'quantiles': [[8.667020766901738]]},
 'IQRPreprocessor1': {'column_names': ['feature_2', 'feature_3'],
  'medians': [7.966513249415563, 26.121139687894964],
  'quartiles': [[3.841161487590153, 15.041517661578098],
   [19.413059393319706, 34.092457050535074]]},
 'IQRPreprocessor2': {'column_names': ['feature_1'],
  'medians': [2.3952925928404154],
  'quartiles': [[1.1107937563733619, 4.123781054770344]]},
 'LogTransformer1': {'column_names': ['feature_1']},
 'BoxCoxTransformer1': {'column_names': ['feature_2', 'feature_3'],
  'lambda_': [0.17890980719038593, 0.33169190917986685]},
 'Cuped1': {'theta': 765.5485835969596, 'bias': 5.852793206114785}}

In [14]:
transformer = Preprocessor(table)

In [15]:
transformer.run_from_config(load_path)

ambrosia LOGGER: Making right-tail robust transformation of columns ['feature_1']
                 with alphas = [0.01]
ambrosia LOGGER: 

ambrosia LOGGER: Change Mean feature_1: 2.8399 ===> 2.7703
ambrosia LOGGER: Change Variance feature_1: 4.3314 ===> 3.8851
ambrosia LOGGER: Change IQR feature_1: 3.0727 ===> 3.0232
ambrosia LOGGER: Change Range feature_1: 12.1095 ===> 8.6641
ambrosia LOGGER: Making IQR transformation of columns ['feature_2', 'feature_3']
ambrosia LOGGER: 

ambrosia LOGGER: Change Mean feature_2: 10.9683 ===> 9.5020
ambrosia LOGGER: Change Variance feature_2: 98.9297 ===> 51.3558
ambrosia LOGGER: Change IQR feature_2: 11.2004 ===> 10.0196
ambrosia LOGGER: Change Range feature_2: 75.3334 ===> 30.7439
ambrosia LOGGER: 

ambrosia LOGGER: Change Mean feature_3: 27.7301 ===> 27.1216
ambrosia LOGGER: Change Variance feature_3: 123.0687 ===> 102.6352
ambrosia LOGGER: Change IQR feature_3: 14.6794 ===> 14.1475
ambrosia LOGGER: Change Range feature_3: 76.8028 ===> 49.1501
ambr

      feature_1  feature_2  feature_3       target
1      1.807905   1.775187   6.774050  2300.801046
2      1.830794   3.363904   4.872551   869.573677
3      1.160506   3.447586   2.887106   157.878257
4      1.575655   3.110634   5.163880  1020.129064
5      1.104943   2.044608   6.420628  1942.369042
...         ...        ...        ...          ...
2995   1.363962   2.932011   7.366038  2903.761007
2996   0.431863   1.704545   5.436949  1168.389537
2997   1.309711   2.341441   8.172211  3883.220463
2998   0.886774   1.263778   5.950709  1543.097011
2999   0.944045   2.174664   4.803001   796.584377

[2774 rows x 4 columns]
[      feature_1  feature_2  feature_3       target
1      1.807905   1.775187   6.774050  2300.801046
2      1.830794   3.363904   4.872551   869.573677
3      1.160506   3.447586   2.887106   157.878257
4      1.575655   3.110634   5.163880  1020.129064
5      1.104943   2.044608   6.420628  1942.369042
...         ...        ...        ...          ...
2995 

TypeError: unhashable type: 'DataFrame'

In [12]:
Preprocessor

ambrosia.preprocessing.preprocessor.Preprocessor

In [12]:
a = 1

In [14]:
globals()['a']

1

In [7]:
import sys

In [16]:
import inspect

In [28]:
"dataframe" in inspect.signature(Preprocessor).parameters

False

In [11]:
sys.modules[__name__]

<module '__main__'>

In [3]:
globals()

['In',
 'NamespaceMagics',
 'Out',
 '_',
 '_2',
 '_Jupyter',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_getcontentof',
 '_getshapeof',
 '_getsizeof',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_ih',
 '_ii',
 '_iii',
 '_nms',
 '_oh',
 'exit',
 'get_ipython',
 'getsizeof',
 'json',
 'np',
 'open',
 'quit',
 'var_dic_list',
 'yapf_reformat']

In [10]:
globals()['Preprocessor']

KeyError: 'Preprocessor'

In [6]:
import sys

In [8]:
sys.modules['Preprocessor']

KeyError: 'Preprocessor'

In [4]:
getattr('Preprocessor')

TypeError: getattr expected at least 2 arguments, got 1

In [4]:
aggregator = AggregatePreprocessor(categorial_method="mode", real_method="mean")

In [10]:
    agg_params = {
        "sessions": "median",
        "gender": "last",
    }

In [11]:
aggregator.fit_transform(table, 'id', agg_params)

,id,sessions,gender
0,0,1.0,Male
1,1,2.0,Female
2,2,3.0,Female
3,3,1.0,Female
4,4,1.0,Female
...,...,...,...
4995,4995,1.0,Male
4996,4996,2.0,Female
4997,4997,2.0,Male
4998,4998,3.0,Female


In [7]:
agg_params['sessions'] = 'kek'

In [12]:
aggregator.agg_params

{'sessions': 'median', 'gender': 'last'}

In [14]:
    aggregator = AggregatePreprocessor(categorial_method="mode", real_method="mean")
    groupby_columns = ["id"]
    agg_params = {
        "sessions": "median",
        "gender": "last",
    }
    aggregator.fit(
        dataframe=table,
        groupby_columns=groupby_columns,
        # agg_params=agg_params,
        real_cols=["sessions"],
        categorial_cols="gender",
    )

In [2]:
from ambrosia.preprocessing import Preprocessor

In [12]:
preproc = Preprocessor(table)

In [28]:
import sys

In [70]:
def kek(lvl):
    print(inspect.stack()[1][3])

In [73]:
kek

<function __main__.kek()>

In [71]:
import inspect

def Kek():
    'hey'
    kek()

In [75]:
''.join(filter(str.isalpha, 'jfdjfsj11'))

'jfdjfsj'

In [72]:
Kek()

Kek


In [ ]:
dict()

In [32]:
sys._getframe()

<frame at 0x7f7c2f468410, file '/var/folders/7_/hzpv_sys2c34hr0hdqhgx26wx_3yvd/T/ipykernel_91670/2963268759.py', line 1, code <module>>

In [31]:
functionNameAsString = sys._getframe().preproc

AttributeError: 'frame' object has no attribute 'preproc'

In [26]:
preproc.transformations()[0].transform()

TypeError: transform() missing 1 required positional argument: 'dataframe'

In [16]:
table

,id,gender,watched,sessions,day,platform
0,0,Male,28.440846,4,1,android
1,1,Female,1.825271,2,1,ios
2,2,Female,46.995606,0,1,web
3,3,Female,37.310264,1,1,ios
4,4,Female,147.513105,0,1,web
...,...,...,...,...,...,...
34995,4995,Male,116.832887,0,7,web
34996,4996,Female,13.558919,2,7,web
34997,4997,Male,24.035386,2,7,android
34998,4998,Female,18.482640,2,7,android


In [15]:
column_names 

NameError: name 'column_names' is not defined

In [17]:
getattr(preproc, 'iqr')(column_names='watched')

ambrosia LOGGER: Making IQR transformation of columns ['watched']
ambrosia LOGGER: 

ambrosia LOGGER: Change Mean watched: 100.2381 ===> 84.9118
ambrosia LOGGER: Change Variance watched: 10070.7557 ===> 5161.8108
ambrosia LOGGER: Change IQR watched: 110.3941 ===> 98.3596
ambrosia LOGGER: Change Range watched: 1251.7118 ===> 304.6217


In [15]:
aggregator.agg_params

{'sessions': 'mean',
 'gender': <function ambrosia.preprocessing.aggregate.AggregatePreprocessor.__mode_calculation(values: pandas.core.series.Series) -> Any>}

In [39]:
df = pd.DataFrame(np.random.normal(size=(5_000, 3)),
                   columns=["metric_val1", "metric_val2", "metric_val3"])

In [ ]:
learned_dict = robust.get_params_dict()

In [5]:
robust.fit_transform(df, ["metric_val1", "metric_val2", "metric_val3"], alpha=np.array([0.05, 0.1, 0.2]), inplace=True)

NameError: name 'robust' is not defined

In [7]:
robust = RobustPreprocessor()

In [30]:
kek = {'kek': 1, 'quantiles': np.array([1,2,3])}

In [28]:
{key: value for key, value in kek.items()}

{'kek': 1, 'lol': 2}

In [5]:
robust.load_params_dict(learned_dict)

NameError: name 'learned_dict' is not defined

In [6]:
df.metric_val1.quantile(0.025)

-1.9732277723005935

In [7]:
df.metric_val1.quantile(0.975)

1.914423531233203

In [23]:
df.metric_val3.quantile(0.)

1.6692246839809284

In [8]:
robust.params

{'tail': None, 'column_names': None, 'alpha': None, 'quantiles': None}

In [17]:
robust.fit(df, ["metric_val1", 'metric_val2'], alpha=np.array([0.05, 0.05]))

In [21]:
robust.params['quantiles'].tolist()

[[-1.985307460766111, 1.9271982876185825],
 [-1.9582737959218812, 1.9601237737405255]]

In [18]:
np.array(robust.params['quantiles'].tolist())

array([[-1.98530746,  1.92719829],
       [-1.9582738 ,  1.96012377]])

In [10]:
robust.params

{'tail': 'both',
 'column_names': ['metric_val1'],
 'alpha': array([0.05]),
 'quantiles': array([[-1.98530746,  1.92719829]])}

In [11]:
df_transformed = robust.transform(df)

ambrosia LOGGER: Making robust transformation of columns ['metric_val1'] 
                 with alphas = [0.05]
ambrosia LOGGER: 

ambrosia LOGGER: Change Mean metric_val1: -0.0070 ===> -0.0045
ambrosia LOGGER: Change Variance metric_val1: 0.9753 ===> 0.7434
ambrosia LOGGER: Change IQR metric_val1: 1.3370 ===> 1.2507
ambrosia LOGGER: Change Range metric_val1: 6.6966 ===> 3.8876


In [12]:
robust.params["quantiles"].shape

(1, 2)

In [13]:
np.zeros((3, 1))

array([[0.],
       [0.],
       [0.]])

In [14]:
kek = np.array([[-1.59370383, -1.32681326, -0.83831528]]).T

In [ ]:
kek[2]

In [ ]:
robust.params["quantiles"]

In [ ]:
robust

In [ ]:
df[robust.params["column_names"]]

In [ ]:
robust.params

In [ ]:
df.loc[robust.mask].index

In [ ]:
robust.mask

In [ ]:
df

In [ ]:
df

In [ ]:
robust.transform(df, inplace=True)

# test IQR

In [7]:
df = pd.DataFrame(np.random.normal(size=(5_000, 3)),
                   columns=["metric_val1", "metric_val2", "metric_val3"])

In [7]:
cols = ["metric_val1", "metric_val2"]

In [8]:
iqrclass = IQRPreprocessor()

In [9]:
iqrclass.fit_transform(df, cols, inplace=False)

ambrosia LOGGER: Making IQR transformation of columns ['metric_val1', 'metric_val2']
ambrosia LOGGER: 

ambrosia LOGGER: Change Mean metric_val1: 0.0148 ===> 0.0107
ambrosia LOGGER: Change Variance metric_val1: 0.9888 ===> 0.9220
ambrosia LOGGER: Change IQR metric_val1: 1.3436 ===> 1.3340
ambrosia LOGGER: Change Range metric_val1: 7.4759 ===> 5.3588
ambrosia LOGGER: 

ambrosia LOGGER: Change Mean metric_val2: -0.0291 ===> -0.0298
ambrosia LOGGER: Change Variance metric_val2: 0.9931 ===> 0.9275
ambrosia LOGGER: Change IQR metric_val2: 1.3460 ===> 1.3364
ambrosia LOGGER: Change Range metric_val2: 8.0403 ===> 5.3635


,metric_val1,metric_val2,metric_val3
0,-1.509132,-1.038221,1.486457
1,-0.388096,0.856851,0.643557
2,0.439795,0.393159,0.969933
3,-0.023978,-1.339821,0.389628
5,-0.062325,0.067058,-0.012226
...,...,...,...
4995,-0.676269,0.082028,0.577783
4996,0.994633,-0.025512,-0.029609
4997,-1.253755,-1.456992,-1.371186
4998,-0.029100,0.315267,-0.172862


In [12]:
df

,metric_val1,metric_val2,metric_val3
0,-0.300136,0.085199,0.992214
1,1.184154,1.214958,1.006129
2,-0.697817,0.368578,-0.584481
3,1.510678,-1.080187,0.125386
4,-1.574673,-0.293700,1.335375
...,...,...,...
4995,0.656436,-1.741118,1.909299
4996,0.209243,-1.161018,0.152228
4997,-0.834028,-0.411020,0.612697
4998,0.846411,0.774942,-1.086685


In [12]:
iqrclass.fit(df, cols)

In [22]:
iqrclass.transform(df, inplace=True)

In [23]:
df

,metric_val1,metric_val2,metric_val3
0,-0.484322,0.643399,0.427435
1,0.023709,-0.338474,0.608964
2,0.634201,0.572607,0.129765
3,1.166159,-0.025467,-0.143979
4,-0.459135,-0.107858,0.366649
...,...,...,...
4995,-0.324446,-0.500512,-0.164044
4996,-2.706544,-1.030363,-1.206979
4997,-0.126194,-1.066703,-0.094655
4998,0.147374,0.450617,-1.784715


In [43]:
iqr = iqrclass.params["quartiles"][:, 1] - iqrclass.params["quartiles"][:, 0]
tail = iqrclass.params["quartiles"][:, 0] - 1.5 * iqr
head = iqrclass.params["quartiles"][:, 1] + 1.5 * iqr

TypeError: 'NoneType' object is not subscriptable

In [27]:
iqrclass.params["quartiles"]

array([[-0.6301564 ,  0.6898051 ],
       [-0.70395877,  0.66930072],
       [-0.65542166,  0.64453047]])

In [28]:
tail

array([-2.61009864, -2.763848  , -2.60534986])

In [29]:
mask = (df[iqrclass.params["column_names"]] < tail) | (df[iqrclass.params["column_names"]] > head)

In [30]:
mask.any(axis=1).sum()

123

In [33]:
iqrclass.mask

0        True
1       False
2        True
3       False
4       False
        ...  
4995    False
4996    False
4997    False
4998    False
4999    False
Length: 5000, dtype: bool

In [34]:
iqrclass.transform(df)

,metric_val1,metric_val2,metric_val3
1,0.720170,0.378941,1.301612
3,-1.056326,1.793595,1.529396
4,-1.076965,-2.172287,-0.020836
5,0.519030,-1.770059,1.857670
6,1.350175,-0.193829,-0.059907
...,...,...,...
4995,0.537855,-2.717513,1.365800
4996,0.113153,0.051330,-0.117054
4997,-0.140213,0.122161,0.113612
4998,-0.883253,-0.601246,-0.464188


In [13]:
iqrclass.get_params_dict()

{'column_names': ['metric_val1', 'metric_val2'],
 'medians': array([0.01412424, 0.00152882]),
 'quartiles': array([[-0.66525203,  0.66603767],
        [-0.67209298,  0.65946556]])}

# test Box Cox

In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
from ambrosia.preprocessing.robust import RobustPreprocessor, IQRPreprocessor, LogTransformer, BoxCoxTransformer

In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.DataFrame(np.random.normal(size=(5_000, 3)),
                   columns=["metric_val1", "metric_val2", "metric_val3"])

In [5]:
import random, string

def randomword(length):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

In [6]:
df['id'] =  df.apply(lambda x: randomword(28), axis=1)

In [7]:
boxcox = BoxCoxTransformer()

In [11]:
boxcox.fit(df, cols)

In [12]:
boxcox.get_params_dict()

{'column_names': ['metric_val1', 'metric_val2'],
 'lambda_': array([0.42821399, 0.42724942])}

In [13]:
df

,metric_val1,metric_val2,metric_val3,id
0,0.240243,0.460883,1.214133,sdithreliyxyxrigwpxlztnhkhhg
1,0.878765,0.567552,0.437754,jhzywvbostfdttbcxpqgmyhstwjs
2,2.128493,2.281123,0.544554,jdgcebydgcqshvpqlyjdvpbcnxic
3,0.683000,0.856081,1.296684,wfyfpxatbyjypyxfthgjjxhbbagf
4,0.459182,0.523063,0.894781,xpctzdgrrwpblegafkwvkixhsjuv
...,...,...,...,...
4995,0.640490,0.891341,1.573491,ozlxnjzuzizoplaiilfxietmsvpf
4996,0.873785,1.633371,0.110024,eghxqeuiijeyduihuzfgexvhjzkv
4997,0.120386,1.822059,0.531115,ygevukwejqkovjyrixrgcbcwxwjd
4998,2.015623,0.762472,2.459815,lslfqksnhzhkiaxsgpnaftmfrxnt


In [20]:
str(boxcox)

'Box-Cox transformation'

In [19]:
boxcox.transform(df, inplace=True)

,metric_val1,metric_val2,metric_val3,id
0,0.388252,-2.261356,0.866494,gtwqhambzeybdnzzuxnzxkmyuoly
1,0.604751,-1.011895,0.452002,uswzsakeexvcsraxaqouazytxrxx
2,-0.950704,0.712862,1.126189,ypdafgtoyewpbyfyrxwdmascgxxq
3,0.127157,-0.769377,2.024744,hmriajnzokurhoenenanoerwvmkc
4,-0.168260,-0.818581,0.221281,uxlkyvxpctlagfoycjvxdwqmlmwh
...,...,...,...,...
4995,-0.449457,-1.571021,0.870054,axlqwcqcwxzivuzreigcvemmpkie
4996,-0.270050,-0.291021,0.251562,ixvfsajffldljubzwfnftmadlpxu
4997,-0.973418,-0.656206,1.967607,wgmszarnirjuoylehkvzhpklspka
4998,0.023323,-0.167547,0.364445,pvpylwqygzhbfmjdkjrmlqnzsdcy


In [40]:
class Kek:
    def __str__(self):
        return'Kek class'
    def __uma(self):
        print("uma")

In [41]:
kek = Kek()

In [42]:
kek.__uma()

AttributeError: 'Kek' object has no attribute '__uma'

In [33]:
import catboost

In [35]:
kek = catboost.CatBoostClassifier()

In [36]:
kek

In [38]:
kek.__str__()

'<catboost.core.CatBoostClassifier object at 0x135ca6b80>'

In [37]:
kek.__repr__()

'<catboost.core.CatBoostClassifier object at 0x135ca6b80>'

In [21]:
print(boxcox)

Box-Cox transformation


In [14]:
boxcox.transform(df, inplace=True)

In [15]:
df

,metric_val1,metric_val2,metric_val3,id
0,-1.067264,-0.659476,1.214133,sdithreliyxyxrigwpxlztnhkhhg
1,-0.125726,-0.503095,0.437754,jhzywvbostfdttbcxpqgmyhstwjs
2,0.891907,0.988628,0.544554,jdgcebydgcqshvpqlyjdvpbcnxic
3,-0.351765,-0.150345,1.296684,wfyfpxatbyjypyxfthgjjxhbbagf
4,-0.661894,-0.566074,0.894781,xpctzdgrrwpblegafkwvkixhsjuv
...,...,...,...,...
4995,-0.405602,-0.112248,1.573491,ozlxnjzuzizoplaiilfxietmsvpf
4996,-0.131097,0.545863,0.110024,eghxqeuiijeyduihuzfgexvhjzkv
4997,-1.392027,0.683877,0.531115,ygevukwejqkovjyrixrgcbcwxwjd
4998,0.817483,-0.256068,2.459815,lslfqksnhzhkiaxsgpnaftmfrxnt


In [16]:
boxcox.inverse_transform(df, inplace=False)

,metric_val1,metric_val2,metric_val3,id
0,0.240243,0.460883,1.214133,sdithreliyxyxrigwpxlztnhkhhg
1,0.878765,0.567552,0.437754,jhzywvbostfdttbcxpqgmyhstwjs
2,2.128493,2.281123,0.544554,jdgcebydgcqshvpqlyjdvpbcnxic
3,0.683000,0.856081,1.296684,wfyfpxatbyjypyxfthgjjxhbbagf
4,0.459182,0.523063,0.894781,xpctzdgrrwpblegafkwvkixhsjuv
...,...,...,...,...
4995,0.640490,0.891341,1.573491,ozlxnjzuzizoplaiilfxietmsvpf
4996,0.873785,1.633371,0.110024,eghxqeuiijeyduihuzfgexvhjzkv
4997,0.120386,1.822059,0.531115,ygevukwejqkovjyrixrgcbcwxwjd
4998,2.015623,0.762472,2.459815,lslfqksnhzhkiaxsgpnaftmfrxnt


In [17]:
boxcox.inverse_transform(df, inplace=True)

# test Log

In [39]:
log = LogTransformer()

In [9]:
cols = ["metric_val1", "metric_val2"]

In [41]:
log.fit(df, cols)

In [42]:
log.get_params_dict()

{'column_names': ['metric_val1', 'metric_val2']}

In [10]:
df[['metric_val1', 'metric_val2', 'metric_val3']] = df[['metric_val1', 'metric_val2', 'metric_val3']].apply(abs)

In [45]:
kek = df.copy()

In [44]:
log.transform(df, inplace =False)

,metric_val1,metric_val2,metric_val3,id
0,-0.947067,0.030203,1.849447,puvfvuqtczdzsenrlhflwnodwewk
1,0.088531,0.446179,1.759319,eapgrxreeojjfkwvosmtulfkoggr
2,-0.437952,-0.414052,1.401652,xcvbippbqfvimjvqatwuawyiryzr
3,0.185472,0.467904,0.923105,uasuodjswzirdhgqcsynkqyviibb
4,-0.633778,0.751230,2.063343,nphigvwfdrjyxcedsdrrbswccgaa
...,...,...,...,...
4995,-3.244132,-0.559503,1.827769,sbbaavqgyvgnzfiwdjjjclhhkfee
4996,-1.569467,-1.602806,0.456478,tvtgfsdkgjgcpyngabnsvjwkapga
4997,-2.438303,-1.700029,1.330066,ofxjsfzeacyrytlfojblbjavrgtx
4998,-5.560738,-0.545777,0.631886,aozygkiuvizgehweiabzmhueukwm


In [46]:
kek

,metric_val1,metric_val2,metric_val3,id
0,0.387877,1.030664,1.849447,puvfvuqtczdzsenrlhflwnodwewk
1,1.092568,1.562332,1.759319,eapgrxreeojjfkwvosmtulfkoggr
2,0.645357,0.660967,1.401652,xcvbippbqfvimjvqatwuawyiryzr
3,1.203787,1.596644,0.923105,uasuodjswzirdhgqcsynkqyviibb
4,0.530583,2.119606,2.063343,nphigvwfdrjyxcedsdrrbswccgaa
...,...,...,...,...
4995,0.039002,0.571493,1.827769,sbbaavqgyvgnzfiwdjjjclhhkfee
4996,0.208156,0.201331,0.456478,tvtgfsdkgjgcpyngabnsvjwkapga
4997,0.087309,0.182678,1.330066,ofxjsfzeacyrytlfojblbjavrgtx
4998,0.003846,0.579392,0.631886,aozygkiuvizgehweiabzmhueukwm


In [52]:
log.inverse_transform(df, inplace =True)

,metric_val1,metric_val2,metric_val3,id
0,0.387877,1.030664,1.849447,puvfvuqtczdzsenrlhflwnodwewk
1,1.092568,1.562332,1.759319,eapgrxreeojjfkwvosmtulfkoggr
2,0.645357,0.660967,1.401652,xcvbippbqfvimjvqatwuawyiryzr
3,1.203787,1.596644,0.923105,uasuodjswzirdhgqcsynkqyviibb
4,0.530583,2.119606,2.063343,nphigvwfdrjyxcedsdrrbswccgaa
...,...,...,...,...
4995,0.039002,0.571493,1.827769,sbbaavqgyvgnzfiwdjjjclhhkfee
4996,0.208156,0.201331,0.456478,tvtgfsdkgjgcpyngabnsvjwkapga
4997,0.087309,0.182678,1.330066,ofxjsfzeacyrytlfojblbjavrgtx
4998,0.003846,0.579392,0.631886,aozygkiuvizgehweiabzmhueukwm


In [47]:
log.transform(df, inplace =True)

In [50]:
np.exp(df[cols])

,metric_val1,metric_val2
0,0.387877,1.030664
1,1.092568,1.562332
2,0.645357,0.660967
3,1.203787,1.596644
4,0.530583,2.119606
...,...,...
4995,0.039002,0.571493
4996,0.208156,0.201331
4997,0.087309,0.182678
4998,0.003846,0.579392


In [13]:
df

,metric_val1,metric_val2,metric_val3,id
0,1.411043,-1.475124,-0.561522,fcdwpzjyjlokbveipfhddswvalbm
1,-0.320027,1.843124,1.044989,wlfgadgjbxkeqqshjjokzxmpvspz
2,-0.281602,-1.005838,0.713199,tfimshlctvqsdmoirrzbvfprkofp
3,-0.441225,0.834288,-1.271434,dmoisglgxonxtmdynpkholbpelri
4,0.446221,0.580241,0.626757,vfkqjzlkwukqioswovcsozclofoy
...,...,...,...,...
4995,0.772888,-0.496471,-1.158996,kmsevifizjajfufyefoaykolxgzz
4996,-0.256224,0.841059,-0.084492,tnnvbyzkzfbcvgwsuogpazcxmicr
4997,0.022732,-0.749928,-0.811311,pabrefgvoltihqjqpitesvqyyrer
4998,0.034546,0.980446,-0.139998,bxjdzdkitdtjpwqleizagjruuwfv


# test Preprocessor

In [5]:
from ambrosia.preprocessing import Preprocessor

In [31]:
robust = RobustPreprocessor()

In [32]:
robust.fit(df, 'metric_val1')

In [18]:
df[['metric_val1', 'metric_val2', 'metric_val3']] = np.abs(df[['metric_val1', 'metric_val2', 'metric_val3']])

In [33]:
preprocessor = Preprocessor(df)

In [34]:
df.metric_val1.mean()

0.7962473676244292

In [35]:
res = preprocessor.robust('metric_val1', alpha=0.1).iqr(['metric_val2']).boxcox(['metric_val1', 'metric_val3']).log('metric_val2').data()

ambrosia LOGGER: Making robust transformation of columns ['metric_val1']
                 with alphas = [0.1]
ambrosia LOGGER: 

ambrosia LOGGER: Change Mean metric_val1: 0.7962 ===> 0.7531
ambrosia LOGGER: Change Variance metric_val1: 0.3689 ===> 0.2366
ambrosia LOGGER: Change IQR metric_val1: 0.8471 ===> 0.7507
ambrosia LOGGER: Change Range metric_val1: 3.6997 ===> 1.9026
ambrosia LOGGER: Making IQR transformation of columns ['metric_val2']
ambrosia LOGGER: 

ambrosia LOGGER: Change Mean metric_val2: 0.7761 ===> 0.7420
ambrosia LOGGER: Change Variance metric_val2: 0.3428 ===> 0.2833
ambrosia LOGGER: Change IQR metric_val2: 0.7899 ===> 0.7678
ambrosia LOGGER: Change Range metric_val2: 3.8766 ===> 2.2812


In [33]:
preprocessor.transformers[0]

Robust preprocessing

In [42]:
trans = preprocessor.transformations()

In [38]:
df

,metric_val1,metric_val2,metric_val3,id
0,0.682369,0.199532,0.296020,xdtvpmetwfqpnvzltoushgfovnzs
1,0.172239,0.475359,0.645409,ftjsnhgfazfvpriilglnydsbevoi
2,0.538216,1.227512,0.572236,fgwkdymocqrtozztajenjvhwukox
3,1.966529,0.329519,0.680655,snkoqngjdxyobcdzmyhsggrxepnf
4,1.384287,0.915573,0.866718,oggltsvznzcuufqnkyfvsmjkrpio
...,...,...,...,...
4995,0.988831,0.024259,0.119583,fypjhldbettrkndlvpqxrxtbzoek
4996,0.257089,0.304122,0.317481,imjnymldlzetebgxsmrijfuffdav
4997,1.827396,1.063072,1.796107,agcpvdfveyaalrfgzfndslkcnpgn
4998,0.145532,0.352904,0.119226,baulbiwvdowsihquotvfwltrxoll


In [37]:
data = preprocessor.data()

In [46]:
trans[2].inverse_transform(trans[3].inverse_transform(data))

,metric_val1,metric_val2,metric_val3,id
0,0.682369,0.199532,0.296020,xdtvpmetwfqpnvzltoushgfovnzs
1,0.172239,0.475359,0.645409,ftjsnhgfazfvpriilglnydsbevoi
2,0.538216,1.227512,0.572236,fgwkdymocqrtozztajenjvhwukox
3,1.966529,0.329519,0.680655,snkoqngjdxyobcdzmyhsggrxepnf
4,1.384287,0.915573,0.866718,oggltsvznzcuufqnkyfvsmjkrpio
...,...,...,...,...
4995,0.988831,0.024259,0.119583,fypjhldbettrkndlvpqxrxtbzoek
4996,0.257089,0.304122,0.317481,imjnymldlzetebgxsmrijfuffdav
4997,1.827396,1.063072,1.796107,agcpvdfveyaalrfgzfndslkcnpgn
4998,0.145532,0.352904,0.119226,baulbiwvdowsihquotvfwltrxoll


In [39]:
data

,metric_val1,metric_val2,metric_val3,id
0,-0.352781,-1.611780,-0.948374,xdtvpmetwfqpnvzltoushgfovnzs
1,-1.239042,-0.743686,-0.399221,ftjsnhgfazfvpriilglnydsbevoi
2,-0.544705,0.204990,-0.496425,fgwkdymocqrtozztajenjvhwukox
3,0.783385,-1.110123,-0.354645,snkoqngjdxyobcdzmyhsggrxepnf
4,0.348709,-0.088205,-0.138743,oggltsvznzcuufqnkyfvsmjkrpio
...,...,...,...,...
4995,-0.011205,-3.718959,-1.393996,fypjhldbettrkndlvpqxrxtbzoek
4996,-1.032165,-1.190327,-0.906216,imjnymldlzetebgxsmrijfuffdav
4997,0.687103,0.061163,0.665710,agcpvdfveyaalrfgzfndslkcnpgn
4998,-1.316081,-1.041560,-1.395198,baulbiwvdowsihquotvfwltrxoll


In [35]:
nonlin_var_table = pd.read_csv('tests/test_data/nonlin_var_table.csv')

In [36]:
nonlin_var_table

,feature_1,feature_2,feature_3,target
0,1.333527,34.184225,24.730372,1378.481590
1,6.097661,4.672146,34.833651,2300.801046
2,6.238838,13.921723,18.163784,869.573677
3,3.191549,14.664789,7.577442,157.878257
4,4.833905,11.858664,20.262644,1020.129064
...,...,...,...,...
2995,3.911659,10.560352,41.579517,2903.761007
2996,1.540124,4.427102,22.371634,1168.389537
2997,3.705102,7.068579,52.096223,3883.220463
2998,2.427286,3.124725,26.727786,1543.097011


In [56]:
kek.append({'puma': (1,2)}, ignore_index=True)

,kek,new_col,puma
0,1.0,"(1, 4)",NaN
1,2.0,"(3, 0)",NaN
2,NaN,NaN,"(1, 2)"


In [61]:
moments_robust = pd.DataFrame(index=["mean", "std"])

In [63]:
logged = nonlin_var_table[['feature_1', 'feature_2', 'feature_3']]

In [66]:
logged

,feature_1,feature_2,feature_3
0,1.333527,34.184225,24.730372
1,6.097661,4.672146,34.833651
2,6.238838,13.921723,18.163784
3,3.191549,14.664789,7.577442
4,4.833905,11.858664,20.262644
...,...,...,...
2995,3.911659,10.560352,41.579517
2996,1.540124,4.427102,22.371634
2997,3.705102,7.068579,52.096223
2998,2.427286,3.124725,26.727786


In [67]:
np.log(logged).mean()

feature_1    0.639320
feature_2    2.009838
feature_3    3.244837
dtype: float64

In [73]:
np.log(logged).std()

feature_1    1.109735
feature_2    0.914309
feature_3    0.405373
dtype: float64

In [77]:
np.vstack([np.log(logged).mean().values, np.log(logged).std().values])

array([[0.63932008, 2.00983826, 3.24483653],
       [1.10973526, 0.91430881, 0.4053728 ]])

In [79]:
moments_robust[['log_feature_1', 'log_feature_2', 'log_feature_3']] = np.vstack([np.log(logged).mean().values, np.log(logged).std().values])

In [80]:
moments_robust

,log_feature_1,log_feature_2,log_feature_3
mean,0.639320,2.009838,3.244837
std,1.109735,0.914309,0.405373


In [82]:
import scipy.stats as sps

In [108]:
moments_robust["boxcox_feature_1"] = sps.boxcox(nonlin_var_table.feature_1)[0].mean(), sps.boxcox(nonlin_var_table.feature_1)[0].std()
moments_robust["boxcox_feature_2"] = sps.boxcox(nonlin_var_table.feature_2)[0].mean(), sps.boxcox(nonlin_var_table.feature_2)[0].std()
moments_robust["boxcox_feature_3"] = sps.boxcox(nonlin_var_table.feature_3)[0].mean(), sps.boxcox(nonlin_var_table.feature_3)[0].std()

In [109]:
moments_robust

,log_feature_1,log_feature_2,log_feature_3,boxcox_feature_1,boxcox_feature_2,boxcox_feature_3
mean,0.639320,2.009838,3.244837,1.046164,2.189527,4.454172
std,1.109735,0.914309,0.405373,1.255637,1.048332,0.730086


In [105]:
BoxCoxTransformer().fit_transform(nonlin_var_table, ['feature_1', 'feature_2', 'feature_3']).std()

feature_1      1.255847
feature_2      1.048506
feature_3      0.730208
target       977.302235
dtype: float64

In [ ]:
np.quantile

In [169]:
quantiles

,feature_1,feature_2,feature_3
0.025,0.112853,1.291637,11.008203
0.975,7.583657,38.170640,54.527149


In [200]:
quantiles = nonlin_var_table[['feature_1', 'feature_2', 'feature_3']].quantile(q=[0.025, 0.975])

In [202]:
feat_1_rob = nonlin_var_table[(nonlin_var_table.feature_1 >= quantiles.feature_1.iloc[0] ) & (nonlin_var_table.feature_1 <= quantiles.feature_1.iloc[1])]
moments_robust['robust_both_0.05_feature_1'] = feat_1_rob.feature_1.mean(), feat_1_rob.feature_1.std()

In [203]:
feat_2_rob = nonlin_var_table[(nonlin_var_table.feature_2 >= quantiles.feature_2.iloc[0] ) & (nonlin_var_table.feature_2 <= quantiles.feature_2.iloc[1])]
moments_robust['robust_both_0.05_feature_2'] = feat_2_rob.feature_2.mean(), feat_2_rob.feature_2.std()

In [206]:
feat_3_rob = feat_1_rob[(feat_1_rob.feature_2 >= quantiles.feature_2.iloc[0] ) & (feat_1_rob.feature_2 <= quantiles.feature_2.iloc[1])]
moments_robust['robust_both_0.05_feature_2after1'] = feat_3_rob.feature_2.mean(), feat_3_rob.feature_2.std()

In [211]:
moments_robust

,log_feature_1,log_feature_2,log_feature_3,boxcox_feature_1,boxcox_feature_2,boxcox_feature_3,robust_both_0.05_feature_1,robust_both_0.05_feature_2,robust_both_0.05_feature_2after1
mean,0.639320,2.009838,3.244837,1.046164,2.189527,4.454172,2.757666,10.257010,10.241291
std,1.109735,0.914309,0.405373,1.255637,1.048332,0.730086,1.845768,7.961768,7.933084


In [214]:
res = booster.fit_transform(nonlin_var_table, ['feature_1', 'feature_2'])

ambrosia LOGGER: Making robust transformation of columns ['feature_1', 'feature_2']
                 with alphas = [0.05 0.05]
ambrosia LOGGER: 

ambrosia LOGGER: Change Mean feature_1: 2.8399 ===> 2.7451
ambrosia LOGGER: Change Variance feature_1: 4.3314 ===> 3.4127
ambrosia LOGGER: Change IQR feature_1: 3.0727 ===> 2.9313
ambrosia LOGGER: Change Range feature_1: 12.1095 ===> 7.4707
ambrosia LOGGER: 

ambrosia LOGGER: Change Mean feature_2: 10.9707 ===> 10.2413
ambrosia LOGGER: Change Variance feature_2: 99.2786 ===> 62.9106
ambrosia LOGGER: Change IQR feature_2: 11.2421 ===> 10.5675
ambrosia LOGGER: Change Range feature_2: 75.3334 ===> 36.8747


In [213]:
res = booster.fit_transform(nonlin_var_table, ['feature_1'])

ambrosia LOGGER: Making robust transformation of columns ['feature_1']
                 with alphas = [0.05]
ambrosia LOGGER: 

ambrosia LOGGER: Change Mean feature_1: 2.8399 ===> 2.7577
ambrosia LOGGER: Change Variance feature_1: 4.3314 ===> 3.4057
ambrosia LOGGER: Change IQR feature_1: 3.0727 ===> 2.9229
ambrosia LOGGER: Change Range feature_1: 12.1095 ===> 7.4707


In [215]:
res.std()

feature_1      1.847700
feature_2      7.933084
feature_3     11.127882
target       977.039638
dtype: float64

In [157]:
np.sqrt(3.4018)

1.8443969204051498

In [156]:
np.std(ddof=1)

TypeError: _std_dispatcher() missing 1 required positional argument: 'a'

In [155]:
nonlin_var_table.std()

feature_1      2.081560
feature_2      9.965525
feature_3     11.131110
target       977.302235
dtype: float64

In [218]:
quantiles = nonlin_var_table[['feature_1', 'feature_2', 'feature_3']].quantile(q=[0.25, 0.75])

In [221]:
quantiles

,feature_1,feature_2,feature_3
0.25,1.132285,3.819827,19.436929
0.75,4.205019,15.061902,34.157467


In [230]:
quantiles.loc['top'] = quantiles.iloc[1] + 1.5* (quantiles.iloc[1] - quantiles.iloc[0])
quantiles.loc['bot'] = quantiles.iloc[0] - 1.5* (quantiles.iloc[1] - quantiles.iloc[0])

In [231]:
quantiles

,feature_1,feature_2,feature_3
0.25,1.132285,3.819827,19.436929
0.75,4.205019,15.061902,34.157467
top,8.814119,31.925013,56.238275
bot,-3.476815,-13.043285,-2.643879


In [239]:
feat_1_iqr = nonlin_var_table[(nonlin_var_table.feature_1 >= quantiles.feature_1.iloc[3] ) & (nonlin_var_table.feature_1 <= quantiles.feature_1.iloc[2])]
moments_robust['iqr_feature_1'] = feat_1_iqr.feature_1.mean(), feat_1_iqr.feature_1.std()

feat_2_iqr = nonlin_var_table[(nonlin_var_table.feature_2 >= quantiles.feature_2.iloc[3] ) & (nonlin_var_table.feature_2 <= quantiles.feature_2.iloc[2])]
moments_robust['iqr_feature_2'] = feat_2_iqr.feature_2.mean(), feat_2_iqr.feature_2.std()

feat_3_iqr = feat_1_iqr[(feat_1_iqr.feature_2 >= quantiles.feature_2.iloc[3] ) & (feat_1_iqr.feature_2 <= quantiles.feature_2.iloc[2])]
moments_robust['iqr_feature_2after1'] = feat_3_iqr.feature_2.mean(), feat_3_iqr.feature_2.std()

In [240]:
moments_robust

,log_feature_1,log_feature_2,log_feature_3,boxcox_feature_1,boxcox_feature_2,boxcox_feature_3,robust_both_0.05_feature_1,robust_both_0.05_feature_2,robust_both_0.05_feature_2after1,iqr_feature_1,iqr_feature_2,iqr_feature_2after1
mean,0.639320,2.009838,3.244837,1.046164,2.189527,4.454172,2.757666,10.257010,10.241291,2.770306,9.483861,9.496623
std,1.109735,0.914309,0.405373,1.255637,1.048332,0.730086,1.845768,7.961768,7.933084,1.971392,7.187309,7.185762


In [241]:
booster = IQRPreprocessor()

In [246]:
booster.fit_transform(nonlin_var_table, ['feature_2']).mean()

ambrosia LOGGER: Making IQR transformation of columns ['feature_2']
ambrosia LOGGER: 

ambrosia LOGGER: Change Mean feature_2: 10.9707 ===> 9.4839
ambrosia LOGGER: Change Variance feature_2: 99.2786 ===> 51.6394
ambrosia LOGGER: Change IQR feature_2: 11.2421 ===> 9.9918
ambrosia LOGGER: Change Range feature_2: 75.3334 ===> 30.8669


feature_1       2.819486
feature_2       9.483861
feature_3      27.866093
target       1690.449716
dtype: float64

In [247]:
moments_robust

SyntaxError: invalid decimal literal (1970904186.py, line 1)

In [325]:
quantiles = nonlin_var_table[['feature_1', 'feature_2'
                             ]].quantile(q=[0.1, 0.9])

feat_1_rob = nonlin_var_table[(nonlin_var_table.feature_1 >= quantiles.feature_1.iloc[0])]
moments_robust['robust_left_0.1_feature_1'] = feat_1_rob.feature_1.mean(), feat_1_rob.feature_1.std()

feat_2_rob = nonlin_var_table[(nonlin_var_table.feature_1 <= quantiles.feature_1.iloc[1])]
moments_robust['robust_right_0.1_feature_1'] = feat_2_rob.feature_1.mean(), feat_2_rob.feature_1.std()

In [255]:
moments_robust.drop(columns='robust_right_0.9_feature_1', inplace=True)

In [326]:
moments_robust

,log_feature_1,log_feature_2,log_feature_3,boxcox_feature_1,boxcox_feature_2,boxcox_feature_3,robust_both_0.05_feature_1,robust_both_0.05_feature_2,robust_both_0.05_feature_2after1,robust_left_0.1_feature_1,robust_right_0.1_feature_1,iqr_feature_1,iqr_feature_2,iqr_feature_2after1
mean,0.639320,2.009838,3.244837,1.046164,2.189527,4.454172,2.757666,10.257010,10.241291,3.130867,2.375393,2.770306,9.483861,9.496623
std,1.109735,0.914309,0.405373,1.255637,1.048332,0.730086,1.845768,7.961768,7.933084,1.991460,1.579760,1.971392,7.187309,7.185762


In [332]:
booster = IQRPreprocessor()
booster.fit_transform(nonlin_var_table, 'feature_1').mean()

ambrosia LOGGER: Making IQR transformation of columns ['feature_1']
ambrosia LOGGER: 

ambrosia LOGGER: Change Mean feature_1: 2.8399 ===> 2.7703
ambrosia LOGGER: Change Variance feature_1: 4.3314 ===> 3.8851
ambrosia LOGGER: Change IQR feature_1: 3.0727 ===> 3.0232
ambrosia LOGGER: Change Range feature_1: 12.1095 ===> 8.6641


feature_1       2.770306
feature_2      10.968254
feature_3      27.730082
target       1678.224275
dtype: float64

In [312]:
kek = booster.fit_transform(nonlin_var_table, ['feature_1'], alpha=0.1).feature_1.mean(), booster.fit_transform(nonlin_var_table, ['feature_1'], alpha=0.1).feature_1.std()

ambrosia LOGGER: Making robust transformation of columns ['feature_1']
                 with alphas = [0.1]
ambrosia LOGGER: 

ambrosia LOGGER: Change Mean feature_1: 2.8399 ===> 3.1309
ambrosia LOGGER: Change Variance feature_1: 4.3314 ===> 3.9644
ambrosia LOGGER: Change IQR feature_1: 3.0727 ===> 2.9454
ambrosia LOGGER: Change Range feature_1: 12.1095 ===> 11.6543
ambrosia LOGGER: Making robust transformation of columns ['feature_1']
                 with alphas = [0.1]
ambrosia LOGGER: 

ambrosia LOGGER: Change Mean feature_1: 2.8399 ===> 3.1309
ambrosia LOGGER: Change Variance feature_1: 4.3314 ===> 3.9644
ambrosia LOGGER: Change IQR feature_1: 3.0727 ===> 2.9454
ambrosia LOGGER: Change Range feature_1: 12.1095 ===> 11.6543


In [320]:
np.isclose(np.array(kek), moments.loc['robust_left_0.1_feature_1'], atol=0.0001).all()

True

In [313]:
kek == tuple(moments.loc['robust_left_0.1_feature_1'])

True

In [317]:
moments.loc['robust_left_0.1_feature_1'].round(2)

mean    3.13
std     1.99
Name: robust_left_0.1_feature_1, dtype: float64

In [327]:
moments

,mean,std
log_feature_1,0.639320,1.109735
log_feature_2,2.009838,0.914309
log_feature_3,3.244837,0.405373
boxcox_feature_1,1.046164,1.255637
boxcox_feature_2,2.189527,1.048332
boxcox_feature_3,4.454172,0.730086
robust_both_0.05_feature_1,2.757666,1.845768
robust_both_0.05_feature_2,10.257010,7.961768
robust_both_0.05_feature_2after1,10.241291,7.933084
robust_left_0.1_feature_1,3.130867,1.991460


In [262]:
moments_robust = moments_robust[['log_feature_1', 'log_feature_2', 'log_feature_3', 'boxcox_feature_1',
       'boxcox_feature_2', 'boxcox_feature_3', 'robust_both_0.05_feature_1',
       'robust_both_0.05_feature_2', 'robust_both_0.05_feature_2after1', 'robust_left_0.1_feature_1', 'robust_right_0.1_feature_1',
       'iqr_feature_1', 'iqr_feature_2', 'iqr_feature_2after1',
       ]]

In [263]:
moments_robust

,log_feature_1,log_feature_2,log_feature_3,boxcox_feature_1,boxcox_feature_2,boxcox_feature_3,robust_both_0.05_feature_1,robust_both_0.05_feature_2,robust_both_0.05_feature_2after1,robust_left_0.1_feature_1,robust_right_0.1_feature_1,iqr_feature_1,iqr_feature_2,iqr_feature_2after1
mean,0.639320,2.009838,3.244837,1.046164,2.189527,4.454172,2.757666,10.257010,10.241291,3.130867,10.882165,2.770306,9.483861,9.496623
std,1.109735,0.914309,0.405373,1.255637,1.048332,0.730086,1.845768,7.961768,7.933084,1.991460,9.839790,1.971392,7.187309,7.185762


In [328]:
moments = moments_robust.T

In [329]:
moments.to_csv('moments.csv', index_label='transf_name')

In [286]:
tuple(moments.loc['log_feature_1'].values)

(0.6393200791700496, 1.1097352611771127)

In [285]:
moments.loc['log_feature_1'].values == 

array([0.63932008, 1.10973526])

In [293]:
moments[['mean']].mean().values

array([5.1859748])

In [298]:
(1, 2) == (3, 4)

False

In [296]:
moments.loc['log_feature_1'].values

array([0.63932008, 1.10973526])

In [282]:
pd.read_csv('moments.csv', index_col=0).loc['log_feature_1']

mean    0.639320
std     1.109735
Name: log_feature_1, dtype: float64

In [1]:
from ambrosia.preprocessing import MLVarianceReducer

In [3]:
MLVarianceReducer(1, 'kek')

AttributeError: 'int' object has no attribute 'columns'

In [10]:
import pandas as pd

In [11]:
from ambrosia.preprocessing import AggregatePreprocessor

In [14]:
table = pd.read_csv('tests/test_data/week_metrics.csv')

In [15]:
# These arguments values are also default ones
transformer = AggregatePreprocessor(categorial_method='mode', real_method='sum')

In [16]:
transformer.run(
    dataframe=table,
    groupby_columns='id',
    real_cols=['watched', 'sessions'],
    categorial_cols=['gender', 'platform']
)

,id,watched,sessions,gender,platform
0,0,772.597224,8,Male,ios
1,1,538.076739,15,Female,android
2,2,288.492353,20,Female,android
3,3,373.620408,9,Female,ios
4,4,630.238862,14,Female,ios
...,...,...,...,...,...
4995,4995,390.133588,14,Male,android
4996,4996,544.423724,25,Female,ios
4997,4997,204.713032,19,Male,android
4998,4998,1088.642872,25,Female,web


In [19]:
transformer.agg_params

{'watched': 'sum',
 'sessions': 'sum',
 'gender': <function ambrosia.preprocessing.aggregate.AggregatePreprocessor.__mode_calculation(values: pandas.core.series.Series) -> Any>,
 'platform': <function ambrosia.preprocessing.aggregate.AggregatePreprocessor.__mode_calculation(values: pandas.core.series.Series) -> Any>}

In [20]:
transformer.transform(dataframe)

NameError: name 'dataframe' is not defined

In [18]:
dir(transformer)

['_AggregatePreprocessor__categorial_case_step',
 '_AggregatePreprocessor__empty_args_step',
 '_AggregatePreprocessor__mode_calculation',
 '_AggregatePreprocessor__real_case_step',
 '_AggregatePreprocessor__simple_agg',
 '_AggregatePreprocessor__transform_agg_param',
 '_AggregatePreprocessor__transform_params',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'agg_params',
 'categorial_method',
 'get_params_dict',
 'groupby_columns',
 'real_method',
 'run',
 'transform']

In [22]:
from typing import Dict

In [23]:
class Borg:
    _shared_state: Dict[str, str] = {}

    def __init__(self) -> None:
        self.__dict__ = self._shared_state


class YourBorg(Borg):
    def __init__(self, state: str = None) -> None:
        super().__init__()
        if state:
            self.state = state
        else:
            # initiate the first instance with default state
            if not hasattr(self, "state"):
                self.state = "Init"

    def __str__(self) -> str:
        return self.state

In [24]:
rm1 = YourBorg()

In [26]:
print(rm1)

Init


In [27]:
rm2 = YourBorg()

In [28]:
print(rm2)

Init


In [30]:
rm1.state = 'No'

In [32]:
rm1.kek = 4

In [31]:
print(rm1)
print(rm2)

No
No


In [33]:
rm1.kek

4

In [34]:
rm2.kek

4

In [37]:
df_1

NameError: name 'df_1' is not defined

In [38]:
dir()

['AggregatePreprocessor',
 'Borg',
 'Dict',
 'In',
 'MLVarianceReducer',
 'NamespaceMagics',
 'Out',
 'YourBorg',
 '_',
 '_16',
 '_17',
 '_18',
 '_19',
 '_25',
 '_33',
 '_34',
 '_4',
 '_5',
 '_6',
 '_Jupyter',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_getcontentof',
 '_getshapeof',
 '_getsizeof',
 '_i',
 '_i1',
 '_i10',
 '_i11',
 '_i12',
 '_i13',
 '_i14',
 '_i15',
 '_i16',
 '_i17',
 '_i18',
 '_i19',
 '_i2',
 '_i20',
 '_i21',
 '_i22',
 '_i23',
 '_i24',
 '_i25',
 '_i26',
 '_i27',
 '_i28',
 '_i29',
 '_i3',
 '_i30',
 '_i31',
 '_i32',
 '_i33',
 '_i34',
 '_i35',
 '_i36',
 '_i37',
 '_i38',
 '_i4',
 '_i5',
 '_i6',
 '_i7',
 '_i8',
 '_i9',
 '_ih',
 '_ii',
 '_iii',
 '_nms',
 '_oh',
 'exit',
 'get_ipython',
 'getsizeof',
 'json',
 'np',
 'open',
 'pd',
 'quit',
 'rm1',
 'rm2',
 'table',
 'transformer',
 'var_dic_list',
 'yapf_reformat']

In [2]:
from abc import ABC, abstractmethod
from typing import Iterable, List

import numpy as np
import pandas as pd
import scipy.stats as sps

import ambrosia.tools._lib._bin_ci_aide as helper_dir
import ambrosia.tools.pvalue_tools as pvalue_pkg
from ambrosia import types

RELATIVE_ABSOLUTE_DELTA_ERROR = ValueError("Choose relative or absolute delta, not both")
ROUND_DIGITS: int = 3


class BinomTwoSampleCI(ABC):
    """
    Implementation of two-sample confidence interval.
    Supported the following types  interval_type:

    X ~ Bin(m, p1)
    Y ~ Bin(n, p2)

    Estimated Delta = p1 - p2
    For more information explore:
    http://stat.wharton.upenn.edu/~lbrown/Papers/2005c%20Confidence%20intervals%20for%
    20the%20two%20sample%20binomial%20distribution%20problem.pdf

    Methods
    -------

        Frequency:
        ----------

        Wald's CI - Using MLE estimation for proportions and
                asymptotic normality

        Yule's CI - Using estimation for Var(p1 - p2), in assumption
                (p1 = p2) and (m ~= n). Could be use for criterion
                H0: p1 = p2   vs   H1: p1 != p2
        Yule's modified CI - analogue for previous one for m != n
        Newcombe's CI - combination of CI for p1 and p2, building CI for difference
        Recentred CI - Using t quantiles

        Bayes:
        ------

        Jeffrey's CI - Using for prior distribution Beta(1/2, 1/2)
        Agresti's CI - Using for prior distribution Beta(1, 1)
        Conjugate Beta - Using for prior Beta(n_success, n_failure)

    """

    __PRECISION: float = 0.0001

    # This class just a container with all confidence intervals.
    # Thats why there is no instantiation.
    @abstractmethod
    def __init__(self):
        pass

    @staticmethod
    def __wald_ci(
        sample_a: int, sample_b: int, a_trials: int, b_trials: int, significance_level: float
    ) -> types.IntervalType:
        """
        Implementation of Wald confidence interval.
        """
        p_a_est = sample_a / a_trials
        p_b_est = sample_b / b_trials
        q_a_est = 1 - p_a_est
        q_b_est = 1 - p_b_est
        delta_estimation = p_b_est - p_a_est

        quantile = sps.norm.ppf(significance_level / 2)
        variation = p_a_est * q_a_est / a_trials + p_b_est * q_b_est / b_trials
        shift = -quantile * np.sqrt(variation)

        return delta_estimation - shift, delta_estimation + shift

    @staticmethod
    def __yule_ci(
        sample_a: int, sample_b: int, a_trials: int, b_trials: int, significance_level: float, modified: bool = False
    ) -> types.IntervalType:
        """
        Implementation of Yule's confidence intervals.
        """

        p_a_est = sample_a / a_trials
        p_b_est = sample_b / b_trials
        delta_estimation = p_b_est - p_a_est

        quantile = sps.norm.ppf(significance_level / 2)

        if modified:
            coef_a = b_trials / a_trials
            coef_b = a_trials / b_trials
            p_hat = (sample_a * coef_a + sample_b * coef_b) / (a_trials + b_trials)
            q_hat = 1 - p_hat
            variation = (1 / a_trials + 1 / b_trials) * p_hat * q_hat
        else:
            p_overline = (sample_a + sample_b) / (a_trials + b_trials)
            q_overline = 1 - p_overline
            variation = (1 / a_trials + 1 / b_trials) * p_overline * q_overline

        shift = -quantile * np.sqrt(variation)
        return delta_estimation - shift, delta_estimation + shift

    @staticmethod
    def __bayes_conjugate_beta(
        sample_a: int,
        sample_b: int,
        a_trials: int,
        b_trials: int,
        n_success: int,
        n_failure: int,
        significance_level: float,
    ) -> types.IntervalType:
        """
        Implementation of bayes confidence intervals with cojugate distribution
        Beta(n_success, n_falure).
        """

        p_tilde_a = (sample_a + n_success) / (n_success + n_failure + a_trials)
        p_tilde_b = (sample_b + n_success) / (n_success + n_failure + b_trials)

        q_tilde_a = 1 - p_tilde_a
        q_tilde_b = 1 - p_tilde_b
        delta_tilde = p_tilde_b - p_tilde_a

        quantile = sps.norm.ppf(significance_level / 2)
        variation = p_tilde_a * q_tilde_a / a_trials + p_tilde_b * q_tilde_b / b_trials
        shift = -quantile * np.sqrt(variation)
        return delta_tilde - shift, delta_tilde + shift

    @staticmethod
    def __square_eq_newcombe(p_est: float, m: int, quantile: float) -> types.IntervalType:  # pylint: disable=C0103
        """
        Helper function for newcombe_ci.
        """

        coef_a = 1 + quantile**2 / m
        coef_b = -(2 * p_est + quantile**2 / m)
        coef_c = p_est**2
        d = coef_b**2 - 4 * coef_a * coef_c  # pylint: disable=C0103
        left = (-coef_b - np.sqrt(d)) / (2 * coef_a)
        right = (-coef_b + np.sqrt(d)) / (2 * coef_a)

        return left, right

    @staticmethod
    def __newcombe_ci(
        sample_a: int,
        sample_b: int,
        a_trials: int,
        b_trials: int,
        significance_level: float = 0.05,
    ) -> types.IntervalType:
        """
        Implementation of Newcombe's confidence intervals.
        """
        p_a_est = sample_a / a_trials
        p_b_est = sample_b / b_trials
        delta_est = p_b_est - p_a_est

        quantile = sps.norm.ppf(significance_level / 2)

        # Calculate intevalls for A and B group
        left_bound_group_a, right_bound_group_a = BinomTwoSampleCI.__square_eq_newcombe(p_a_est, a_trials, quantile)
        left_bound_group_b, right_bound_group_b = BinomTwoSampleCI.__square_eq_newcombe(p_b_est, b_trials, quantile)

        variance_a_left = left_bound_group_a * (1 - left_bound_group_a) / a_trials
        variance_a_right = right_bound_group_a * (1 - right_bound_group_a) / a_trials
        variance_b_left = left_bound_group_b * (1 - left_bound_group_b) / b_trials
        variance_b_right = right_bound_group_b * (1 - right_bound_group_b) / b_trials
        variance_left = variance_a_left + variance_b_right
        variance_right = variance_b_left + variance_a_right
        left_bound = delta_est + quantile * np.sqrt(variance_left)
        right_bound = delta_est - quantile * np.sqrt(variance_right)

        return left_bound, right_bound

    @staticmethod
    def __recentered_ci(
        sample_a: int, sample_b: int, a_trials: int, b_trials: int, significance_level: float
    ) -> types.IntervalType:
        """
        Implementation of recentered confidence intervals.
        """
        p_a_est = sample_a / a_trials
        p_b_est = sample_b / b_trials
        delta_estimation = p_b_est - p_a_est
        p_est = (b_trials * p_a_est + a_trials * p_b_est) / (a_trials + b_trials)
        q_est = 1 - p_est
        quantile = -sps.t.ppf(significance_level / 2, df=a_trials + b_trials - 2)
        coef = 1 + quantile**2 / (a_trials + b_trials)
        center = delta_estimation / coef
        var_est = (1 / a_trials + 1 / b_trials) * p_est * q_est
        shift = quantile * np.sqrt(coef * var_est - delta_estimation**2 / (a_trials + b_trials)) / coef
        return center - shift, center + shift

    @staticmethod
    def calculate_pvalue(
        a_success: int,
        b_success: int,
        a_trials: int,
        b_trials: int,
        interval_type: str = "wald",
        alternative: str = "two-sided",
        n_success_conjugate: int = None,
        n_failure_conjugate: int = None,
    ) -> float:
        """
        Calculate pvalue for confidence interval.
        pvalue(x) = inf_a {a | x \\in S_a }

        Parameters
        ----------
        a_success : int
            Samples from Bin(a_trials, p_a)
        b_success : int
            Sample from Bin(b_trials, p_b)
        a_trials : int
            Parameter for trials amount for group A
        b_trials : int
            Parameter for trials amount for group B
        confidence_level : float
            Pr ( [p1 - p2] in CI ) -> confidence_level, n -> infty
        interval_type : str, default : ``"wald"``
            One from [wald, yule, yule_modif, newcombe, jeffrey, agresti, bayes_beta, recenter]
        alternative : str, default : ``"two-sided"``
            Alternative for static criteria - two-sided, less, greater
            Less means, that mean in first group less, than mean in second group
        n_success : int
            Arguments for conjugate distribution Beta(n_success, n_failure) if interval_type = "bayes_beta"
        n_failure : int
            Arguments for conjugate distribution Beta(n_success, n_failure) if interval_type = "bayes_beta"

        Comment
        -------
        You can pass numpy arrays with same shapes for a_success, b_success and e.t.c

        Returns
        -------
        pvalue : float
            P-value of the interval-induced criterion
        """

        return pvalue_pkg.calculate_pvalue_by_interval(
            BinomTwoSampleCI.confidence_interval,
            0,
            a_success=a_success,
            a_trials=a_trials,
            b_success=b_success,
            b_trials=b_trials,
            n_success_conjugate=n_success_conjugate,
            n_failure_conjugate=n_failure_conjugate,
            interval_type=interval_type,
            alternative=alternative,
        )

    @staticmethod
    def confidence_interval(
        a_success: int,
        b_success: int,
        a_trials: int,
        b_trials: int,
        confidence_level: float,
        interval_type: str = "wald",
        alternative: str = "two-sided",
        n_success_conjugate: int = None,
        n_failure_conjugate: int = None,
    ) -> types.IntervalType:
        """
        Main function building confidence interval.

        Parameters
        ----------
        a_success : int
            Samples from Bin(a_trials, p_a)
        b_success : int
            Sample from Bin(b_trials, p_b)
        a_trials : int
            Parameter for trials amount for group A
        b_trials : int
            Parameter for trials amount for group B
        confidence_level : float
            Pr ( [p1 - p2] in CI ) -> confidence_level, n -> infty
        interval_type : str, default : ``"wald"``
            One from [wald, yule, yule_modif, newcombe, jeffrey, agresti, bayes_beta, recenter]
        alternative : str, default : ``"two-sided"``
            Alternative for static criteria - two-sided, less, greater
            Less means, that mean in first group less, than mean in second group
        n_success : int
            Arguments for conjugate distribution Beta(n_success, n_failure) if interval_type = "bayes_beta"
        n_failure : int
            Arguments for conjugate distribution Beta(n_success, n_failure) if interval_type = "bayes_beta"

        Note
        ----
        You can pass numpy arrays with same shapes for a_success, b_success and e.t.c

        Returns
        -------
        interval : Tuple[float, float]
        """

        valid_types: List[str] = [
            "wald",
            "yule",
            "yule_modif",
            "newcombe",
            "jeffrey",
            "agresti",
            "bayes_beta",
            "recenter",
        ]
        pvalue_pkg.check_alternative(alternative)
        significance_level = pvalue_pkg.corrected_alpha(1 - confidence_level, alternative)
        if interval_type == "wald":
            left_ci, right_ci = BinomTwoSampleCI.__wald_ci(a_success, b_success, a_trials, b_trials, significance_level)
        elif interval_type == "yule":
            left_ci, right_ci = BinomTwoSampleCI.__yule_ci(
                a_success, b_success, a_trials, b_trials, significance_level, modified=False
            )
        elif interval_type == "yule_modif":
            left_ci, right_ci = BinomTwoSampleCI.__yule_ci(
                a_success, b_success, a_trials, b_trials, significance_level, modified=True
            )
        elif interval_type == "newcombe":
            left_ci, right_ci = BinomTwoSampleCI.__newcombe_ci(
                a_success, b_success, a_trials, b_trials, significance_level
            )
        elif interval_type == "recenter":
            left_ci, right_ci = BinomTwoSampleCI.__recentered_ci(
                a_success, b_success, a_trials, b_trials, significance_level
            )
        elif interval_type == "jeffrey":
            left_ci, right_ci = BinomTwoSampleCI.__bayes_conjugate_beta(
                a_success, b_success, a_trials, b_trials, 0.5, 0.5, significance_level
            )
        elif interval_type == "agresti":
            left_ci, right_ci = BinomTwoSampleCI.__bayes_conjugate_beta(
                a_success, b_success, a_trials, b_trials, 1, 1, significance_level
            )
        elif interval_type == "bayes_beta":
            error_beta_params: str = "Pass correct params n_success_conjugate, n_faliure_conjugate"
            args_correctness = (
                (n_success_conjugate is not None)
                and (n_failure_conjugate is not None)
                and (n_success_conjugate > 0)
                and (n_failure_conjugate > 0)
            )
            if not args_correctness:
                raise ValueError(error_beta_params)
            left_ci, right_ci = BinomTwoSampleCI.__bayes_conjugate_beta(
                a_success, b_success, a_trials, b_trials, n_success_conjugate, n_failure_conjugate, significance_level
            )
        else:
            algo_type_error: str = f'Choose one from accepted methods, from - {", ".join(valid_types)}'
            raise ValueError(algo_type_error)
        left_ci, right_ci = pvalue_pkg.choose_from_bounds(
            left_ci, right_ci, alternative, left_bound=np.array([-1]), right_bound=np.array([1])
        )
        return left_ci, right_ci

In [3]:
def get_table_power_on_size_and_delta(
    p_a: float,
    sample_sizes: Iterable[int],
    interval_type: str = "wald",
    delta_values: Iterable[float] = None,
    delta_relative_values: Iterable[float] = None,
    amount: int = 10000,
    confidence_level: float = 0.95,
) -> pd.DataFrame:
    """
    Table with power / empirical 1 type error = 1 - coverage for fixed size and effect.

    Parameters
    ----------
    interval_type : str
        interval_type for confidence interval
    p_a : Iterable[float]
        Conversion in A group
    sample_sizes : Iterable[float]
        Sizes for samples
    delta_values : Iterable[float]
        Absolute delta values: p_a - p_b = delta
    delta_relative_values : Iterable[float]
        Relative delta values: delta_relative * p_a = p_b
    amount : int, default : ``10000``
        Amount of generated samples for one n(trials amount), to estimate power
    confidence_level : float, default : ``0.95``
        Such value x, that: Pr ( delta in I ) >= x

    Returns
    -------
    table : pd.DataFrame
        Required table with power
    """
    trials = np.array(sample_sizes)
    if not (delta_values is None) ^ (delta_relative_values is None):
        raise RELATIVE_ABSOLUTE_DELTA_ERROR
    if delta_values is not None:
        p_b_values: np.ndarray = p_a - np.array(delta_values)
    else:
        p_b_values: np.ndarray = p_a * np.array(delta_relative_values)
    if not np.all((p_b_values >= 0) & (p_b_values <= 1)):
        raise ValueError(f"Probability of success in group B must be positive, not {p_b_values}")
    powers_array: List[np.ndarray] = []
    sample_a = sps.binom.rvs(n=trials, p=p_a, size=(amount, trials.shape[0]))

    for p_b in p_b_values:
        sample_b = sps.binom.rvs(n=trials, p=p_b, size=(amount, trials.shape[0]))
        binom_kwargs = {
            "interval_type": interval_type,
            "a_success": sample_a,
            "b_success": sample_b,
            "a_trials": trials,
            "b_trials": trials,
            "confidence_level": confidence_level,
        }
        conf_interval: types.ManyIntervalType = BinomTwoSampleCI.confidence_interval(**binom_kwargs)
        power: np.ndarray = helper_dir.__helper_calc_empirical_power(conf_interval)
        powers_array.append(power)
    power_matrix = np.vstack(powers_array)
    index = delta_values if delta_values is not None else delta_relative_values
    table = pd.DataFrame(power_matrix, index=index, columns=sample_sizes)
    table.index.name = r"$\Delta$-absolute" if delta_values is not None else r"$\delta$-relative"
    table.columns.name = "sample sizes"
    table_title: str = r"$1 - \beta$: power of criterion, " + (
        r"$p_a-p_b=\Delta$" if delta_values else r"$p_a\delta=p_b$"
    )
    table = table.style.set_caption(table_title)
    return table

In [4]:
def iterate_for_delta(
    interval_type: str,
    first_errors: Iterable[float],
    second_errors: Iterable[float],
    sample_sizes: Iterable[int],
    p_a: float,
    amount: int,
    delta_type: str,
) -> pd.DataFrame:
    """
    Helps find effect for different params
    """
    values = [(round(a, ROUND_DIGITS), round(b, ROUND_DIGITS)) for a in first_errors for b in second_errors]
    table: pd.DataFrame = pd.DataFrame(
        index=pd.MultiIndex.from_tuples(values, names=[r"$\alpha$", r"$\beta$"]),
        columns=sample_sizes,
    )
    for alpha in first_errors:
        for second_error in second_errors:
            power = 1 - second_error
            for trials in sample_sizes:
                delta = helper_dir.__helper_bin_search_for_delta(
                    interval_type=interval_type,
                    confidence_level=1 - alpha,
                    p_a=p_a,
                    trials=trials,
                    amount=amount,
                    power=power,
                )
                if delta is not None and delta_type == "relative":
                    delta = str(round(abs(delta) / p_a * 100, 2)) + "%"

                table.loc[(alpha, second_error), trials] = delta
    return table

In [20]:
def get_table_power_on_size_and_delta(
    p_a: float,
    sample_sizes: Iterable[int],
    first_errors: Iterable[float] = (0.05,),
    delta_values: Iterable[float] = None,
    delta_relative_values: Iterable[float] = None,
    interval_type: str = "wald",
    alternative: str = "two-sided",
    amount: int = 10000,
) -> pd.DataFrame:
    """
    Table with power / empirical 1 type error = 1 - coverage for fixed size and effect.

    Parameters
    ----------
    p_a : Iterable[float]
        Conversion in A group
    sample_sizes : Iterable[float]
        Sizes for samples
    first_errors : Iterable[float], default : ``(0.05,)``
        First type error values
    delta_values : Iterable[float]
        Absolute delta values: p_a - p_b = delta
    delta_relative_values : Iterable[float]
        Relative delta values: delta_relative * p_a = p_b
    interval_type : str
        interval_type for confidence interval
    alternative : str, default : ``"two-sided"``
        Alternative for static criteria - two-sided, less, greater
        Less means, that mean in first group less, than mean in second group
    amount : int, default : ``10000``
        Amount of generated samples for one n(trials amount), to estimate power

    Returns
    -------
    table : pd.DataFrame
        Required table with power
    """
    trials = np.array(sample_sizes)
    if not (delta_values is None) ^ (delta_relative_values is None):
        raise RELATIVE_ABSOLUTE_DELTA_ERROR
    if delta_values is not None:
        p_b_values: np.ndarray = p_a - np.array(delta_values)
        grid_delta: np.ndarray = delta_values
        delta_name: str = r"$\Delta$-absolute"
    else:
        p_b_values: np.ndarray = p_a * np.array(delta_relative_values)
        grid_delta: np.ndarray = delta_relative_values
        delta_name: str = r"$\delta$-relative"
    if not np.all((p_b_values >= 0) & (p_b_values <= 1)):
        raise ValueError(f"Probability of success in group B must be positive, not {p_b_values}")

    values = [(round(a, ROUND_DIGITS), b) for a in first_errors for b in grid_delta]
    table: pd.DataFrame = pd.DataFrame(
        index=pd.MultiIndex.from_tuples(values, names=[r"$\alpha$", delta_name]),
        columns=trials,
    )
    table.columns.name = "sample sizes"
    sample_a = sps.binom.rvs(n=trials, p=p_a, size=(amount, trials.shape[0]))
    for alpha in first_errors:
        for p_b, delta in zip(p_b_values, grid_delta):
            sample_b = sps.binom.rvs(n=trials, p=p_b, size=(amount, trials.shape[0]))
            binom_kwargs = {
                "interval_type": interval_type,
                "a_success": sample_a,
                "b_success": sample_b,
                "a_trials": trials,
                "b_trials": trials,
                "confidence_level": 1 - alpha,
                "alternative": alternative,
            }
            conf_interval: types.ManyIntervalType = BinomTwoSampleCI.confidence_interval(**binom_kwargs)
            #return conf_interval
            power: np.ndarray = helper_dir.__helper_calc_empirical_power(conf_interval)
            print(power)
            table.loc[(alpha, delta), trials] = [str(round(power_val * 100, 1)) + "%" for power_val in power]
    table_title: str = r"$1 - \beta$: power of criterion, " + (
        r"$p_a-p_b=\Delta$" if delta_values else r"$p_a\delta=p_b$"
    )
    table = table.style.set_caption(table_title)
    return table

In [27]:
from scipy.stats import ttest_ind

In [28]:
ttest_ind([100, 2003], [12030, 5959], alternative='kek')

ValueError: alternative must be 'less', 'greater' or 'two-sided'

In [14]:
interval1

NameError: name 'interval1' is not defined

In [70]:
interval1.__len__()

2

In [68]:
interval1 = get_table_power_on_size_and_delta(0.5, sample_sizes= [100, 200, 500], delta_relative_values = [1.1, 1.01], first_errors=[0.1, 0.2], alternative='two-sided')

In [69]:
interval2 = get_table_power_on_size_and_delta(0.5, sample_sizes= [100, 200, 500], delta_relative_values = [1.1, 1.01], first_errors=[0.1, 0.2], alternative='less')

In [25]:
get_table_power_on_size_and_delta(0.5, sample_sizes= [100, 200], delta_values = [0.1, 0.05], first_errors=[0.1, 0.05])

[0.4146 0.6419]
[0.188  0.2568]
[0.3075 0.5162]
[0.1177 0.1721]


In [30]:
get_table_power_on_size_and_delta(0.5, sample_sizes= [100, 200], delta_values = [0.1, 0.05], confidence_level=0.8)

sample sizes,100,200
$\Delta$-absolute,,
0.1,0.565800,0.767500
0.05,0.296900,0.405700
